In [14]:
import pvl, struct, quaternion
import numpy as np

In [37]:
"""
Read the binary data from an ISIS3 double table and return it as a dictionary

label : The ISIS3 Table label
file : The ISIS3 cube file
"""
def read_table(label, file):
    data_types = {
        'Integer' : {'format':'i', 'size':4},
        'Double'  : {'format':'d', 'size':8},
        'Real'    : {'format':'f', 'size':4},
        'Text'    : {'format':'c', 'size':1}
    }
    with open(file, 'rb') as f:
        f.seek(int(label['StartByte'])-1) # Why is this -1 needed? It's taken straight from the ISIS source Blob::ReadData
        data = f.read(int(label['Bytes']))
    fields = label.getlist('Field')
    results = {field['Name']:[] for field in fields}
    offset = 0
    for record in range(label['Records']):
        for field in fields:
            count = field['Size']
            if field['Type'] == 'Text':
                results[field['Name']].append(data[offset:offset+count].decode(encoding='latin_1'))
            else:
                data_format = data_types[field['Type']]['format'] * count
                field_data = struct.unpack_from(data_format, data, offset)
                if len(field_data) == 1:
                    results[field['Name']].append(field_data[0])
                else:
                    results[field['Name']].append(field_data)
            offset += data_types[field['Type']]['size'] * count
    return results

In [20]:
def read_position_table(label, file):
    bin_data = read_table(label, file)
    results = {}
    if all (key in bin_data for key in ('J2000X','J2000Y','J2000Z')):
        results['Positions'] = [bin_data['J2000X'],bin_data['J2000Y'],bin_data['J2000Z']]
    if 'ET' in bin_data:
        results['Times'] = bin_data['ET']
    if all (key in bin_data for key in ('J2000XV','J2000YV','J2000ZV')):
        results['Velocities'] = [bin_data['J2000XV'],bin_data['J2000YV'],bin_data['J2000ZV']]
    if all (key in bin_data for key in ('J2000SVX','J2000SVY','J2000SVZ')):
        results['PositionCoefficients'] = [bin_data['J2000SVX'][:-1],bin_data['J2000SVY'][:-1],bin_data['J2000SVZ'][:-1]]
        results['BaseTime'] = bin_data['J2000SVX'][-1]
        results['TimeScale'] = bin_data['J2000SVY'][-1]
    return results

In [31]:
def read_rotation_table(label, file):
    bin_data = read_table(label, file)
    results = {}
    if all (key in bin_data for key in ('J2000Q0','J2000Q1','J2000Q2','J2000Q3')):
        results['Rotations'] = [bin_data['J2000Q0'],bin_data['J2000Q1'],bin_data['J2000Q2'],bin_data['J2000Q3']]
    if all (key in bin_data for key in ('AV1','AV1','AV1')):
        results['AngularVelocities'] = [bin_data['AV1'],bin_data['AV1'],bin_data['AV1']]
    if 'ET' in bin_data:
        results['Times'] = bin_data['ET']
    if all (key in bin_data for key in ('J2000Ang1','J2000Ang2','J2000Ang3')):
        results['EulerCoefficients'] = [bin_data['J2000Ang1'],bin_data['J2000Ang2'],bin_data['J2000Ang3']]
        results['BaseTime'] = bin_data['J2000Ang1'][-1]
        results['TimeScale'] = bin_data['J2000Ang2'][-1]
    if 'TimeDependentFrames' in label:
        results['TimeDependentFrames'] = label['TimeDependentFrames']
    if all (key in label for key in ('ConstantRotation','ConstantFrames')):
        const_rotation_mat = np.array(label['ConstantRotation'])
        results['ConstantRotation'] = quaternion.from_rotation_matrix(np.reshape(const_rotation_mat, (3, 3)))
        results['ConstantFrames'] = label['ConstantFrames']
    if all (key in label for key in ('PoleRa','PoleDec','PrimeMeridian')):
        results['BodyRotationCoefficients'] = [label['PoleRa'],label['PoleDec'],label['PrimeMeridian']]
    if all (key in label for key in ('PoleRaNutPrec','PoleDecNutPrec','PmNutPrec','SysNutPrec0','SysNutPrec1')):
        results['SatelliteNutationPrecessionCoefficients'] = [label['PoleRaNutPrec'],label['PoleDecNutPrec'],label['PmNutPrec']]
        results['PlanetNutationPrecessionAngleCoefficients'] = [label['SysNutPrec0'],label['SysNutPrec1']]
    return results

In [48]:
cube_file = '/usgs/cpkgs/isis3/testData/isis/src/control/apps/jigsaw/tsts/radar/input/LSZ_00455_1CD_XKU_87S324_V1_S1_Null.cub'

In [43]:
label = pvl.load(cube_file)
tables = label.getlist('Table')
print(tables)

[PVLObject([
  ('Name', 'InstrumentPointing')
  ('StartByte', 136556582)
  ('Bytes', 76480)
  ('Records', 1195)
  ('ByteOrder', 'Lsb')
  ('TimeDependentFrames', [-85000, 1])
  ('ConstantFrames', [-85700, -85000])
  ('ConstantRotation',
   [1.0,
    0.0,
    0.0,
    0.0,
    0.67430238758372,
    0.73845534062588,
    0.0,
    -0.73845534062588,
    0.67430238758372])
  ('CkTableStartTime', 302388277.71971)
  ('CkTableEndTime', 302388516.48528)
  ('CkTableOriginalSize', 26302)
  ('Description', 'Created by spiceinit')
  ('Kernels',
   ['$lro/kernels/ck/moc42_2009213_2009214_v03.bc',
    '$lro/kernels/fk/lro_frames_2010277_v01.tf'])
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q0')
    ('Type', 'Double')
    ('Size', 1)
  ]))
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q1')
    ('Type', 'Double')
    ('Size', 1)
  ]))
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q2')
    ('Type', 'Double')
    ('Size', 1)
  ]))
  ('Field',
   PVLGroup([
    ('Name', 'J2000Q3')
    ('Type', 'Double')
  

In [44]:
read_position_table(tables[1], cube_file)

{'Positions': [[-156.3249467565359, 42.73228116264915, 241.27240232500955],
  [659.679558446114, 692.9027317007598, 717.5515171234637],
  [-1645.1440297221175, -1638.4398876547368, -1611.4422933973376]],
 'Times': [302388277.7197137, 302388397.097959, 302388516.4852824],
 'Velocities': [[1.6627570662146614, 1.6686724581040384, 1.6538810102678987],
  [0.3131519168739245, 0.2428710020222609, 0.16965810979736903],
  [-0.02926015404243728, 0.14146863974779417, 0.31028387905784444]]}

In [45]:
read_rotation_table(tables[0], cube_file)

{'Rotations': [[0.08968230160712044,
   0.0896641027716965,
   0.08964590343489308,
   0.08962730179469151,
   0.08960889883208234,
   0.08959009871684136,
   0.0895716004383254,
   0.08955400325764862,
   0.08953500317354511,
   0.08951749941155,
   0.08949810375998814,
   0.0894802973442161,
   0.0894614985314101,
   0.08944379994476254,
   0.08942550119977834,
   0.08940649956537056,
   0.08938880306806257,
   0.08937030293570007,
   0.08935310255444436,
   0.08933410025485738,
   0.08931630316803725,
   0.08929760170074705,
   0.08927990026335983,
   0.0892616003661825,
   0.08924310226684569,
   0.08922500215089649,
   0.0892061989507734,
   0.0891879998734177,
   0.08917040069341443,
   0.08915199847440321,
   0.08913379897640368,
   0.08911550036651238,
   0.08909720261686092,
   0.08907870061640887,
   0.08906050126311663,
   0.08904220295961116,
   0.08902370234997144,
   0.08900530064402534,
   0.08898649936685817,
   0.08896899978930471,
   0.0889500013370257,
   0.088932100

In [46]:
read_rotation_table(tables[2], cube_file)

{'Rotations': [[0.7826440333667806, 0.7824563583012707],
  [-0.16405498562923956, -0.1640203287271304],
  [0.10859500956940277, 0.1086471728213603],
  [-0.5905602447017484, -0.5908089123871467]],
 'AngularVelocities': [[6.254229311303041e-08, 6.254250623658643e-08],
  [6.254229311303041e-08, 6.254250623658643e-08],
  [6.254229311303041e-08, 6.254250623658643e-08]],
 'Times': [302388277.7197137, 302388516.4852824],
 'TimeDependentFrames': [31006, 1],
 'ConstantRotation': quaternion(0.99999996831339, 7.58574205202439e-07, 0.000190434690476096, 0.000164642860863603),
 'ConstantFrames': [31001, 31007, 31006]}

In [47]:
read_position_table(tables[3], cube_file)

{'Positions': [[-95685606.07044964, -95691232.17676],
  [108444791.12885298, 108440754.79538174],
  [47040830.85666552, 47039064.29084041]],
 'Times': [302388277.7197137, 302388516.4852824],
 'Velocities': [[-23.563677979659296, -23.56293523023269],
  [-16.904262438258016, -16.905750395728546],
  [-7.398403753766077, -7.399088481933278]]}